En este notebook voy a implementar un modelo de regresión logística aummentando los predictores, además de los rendimientos rezagados, con los indicadores RSI, SMA, MACD y BB.  
También se agregan algunas estrategias que se irán explicando en los comentarios

In [28]:
# Se realizan las importaciones necesarias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
plt.style.use("seaborn-v0_8-whitegrid")

In [29]:
# Función para calcular indicadores técnicos
def calculate_technical_indicators(df, price_col="price"):
    # SMA (Simple Moving Average) - Media móvil simple (ventana de 20 periodos)
    df["SMA"] = df[price_col].rolling(window=20).mean()

    # MACD (Moving Average Convergence Divergence)
    exp1 = df[price_col].ewm(span=12, adjust=False).mean()  # EMA rápida
    exp2 = df[price_col].ewm(span=26, adjust=False).mean()  # EMA lenta
    df["MACD"] = exp1 - exp2
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()  # Línea de señal

    # RSI (Relative Strength Index)
    delta = df[price_col].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # Bandas de Bollinger (ventana de 20 periodos, 2 desviaciones estándar)
    df["BB_Middle"] = df[price_col].rolling(window=20).mean()
    df["BB_Std"] = df[price_col].rolling(window=20).std()
    df["BB_Upper"] = df["BB_Middle"] + 2 * df["BB_Std"]
    df["BB_Lower"] = df["BB_Middle"] - 2 * df["BB_Std"]

    return df


In [30]:
# función para cargar los datos
def carga_archivo(archivo, tipo):
    data = pd.read_csv(archivo)
    if tipo == "b":
        data = data.rename(columns = {"Close time": "time", "Close": "price"})
    data["time"] = pd.to_datetime(data["time"])
    data.set_index("time", inplace=True)
    return data


In [31]:
# seleccionar el archivo de datos
archivo = "EURUSDT_2020_5min.csv"
tipo = "b"

In [32]:
"""Cargar el archivo de datos
Voy a realizar cálculos con diferentes archivos de datos que están en el directorio actual y corresponden a
datos del tutorial y a datos de Binance.
Los datos del tutorial están en el archivo "five_minute.csv" y los datos de Binance están en  "EURUSDT_2020_5MIN.csv" """

df = carga_archivo(archivo, tipo)


In [33]:
df

,price
time,
2020-01-03 08:04:59.999,1.1188
2020-01-03 08:09:59.999,1.1188
2020-01-03 08:14:59.999,1.1188
2020-01-03 08:19:59.999,1.1196
2020-01-03 08:24:59.999,1.1196
...,...
2020-12-30 23:44:59.999,1.2311
2020-12-30 23:49:59.999,1.2310
2020-12-30 23:54:59.999,1.2309


In [34]:
# Calcular los rendimientos logarítmicos
df["returns"] = np.log(df["price"] / df["price"].shift(1))


In [35]:
# Calcular la dirección del mercado (tres clases: +1, 0, -1)
df["direction"] = np.sign(df["returns"])

In [36]:
# Verificar la distribución de clases
print("\nDistribución de clases en 'direction':")
print(df["direction"].value_counts())


Distribución de clases en 'direction':
direction
 0.0    36592
 1.0    33987
-1.0    33616
Name: count, dtype: int64


In [37]:
# Calcular indicadores técnicos
df = calculate_technical_indicators(df)


In [38]:
# Crear cinco predictores de rendimientos rezagados
lags = 5
cols = []
for lag in range(1, lags + 1):
    col = f"lag{lag}"
    df[col] = df["returns"].shift(lag)
    cols.append(col)


In [39]:
# Agregar indicadores técnicos como predictores
technical_indicators = ["SMA", "MACD", "MACD_Signal", "RSI", "BB_Upper", "BB_Lower"]
cols.extend(technical_indicators)

In [40]:
# Eliminar filas con valores NaN
df.dropna(inplace=True)

In [41]:
df

,price,returns,direction,SMA,MACD,MACD_Signal,RSI,BB_Middle,BB_Std,BB_Upper,BB_Lower,lag1,lag2,lag3,lag4,lag5
time,,,,,,,,,,,,,,,,
2020-01-03 09:39:59.999,1.1175,-0.000626,-1.0,1.118520,-0.000222,-0.000130,66.000000,1.118520,0.000838,1.120197,1.116843,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-03 09:44:59.999,1.1175,0.000000,0.0,1.118455,-0.000265,-0.000157,5.555556,1.118455,0.000865,1.120186,1.116724,-0.000626,0.000000,0.000000,0.000000,0.000000
2020-01-03 09:49:59.999,1.1175,0.000000,0.0,1.118390,-0.000296,-0.000185,5.555556,1.118390,0.000887,1.120163,1.116617,0.000000,-0.000626,0.000000,0.000000,0.000000
2020-01-03 09:54:59.999,1.1175,0.000000,0.0,1.118325,-0.000317,-0.000211,0.000000,1.118325,0.000903,1.120130,1.116520,0.000000,0.000000,-0.000626,0.000000,0.000000
2020-01-03 09:59:59.999,1.1151,-0.002150,-1.0,1.118100,-0.000521,-0.000273,0.000000,1.118100,0.001106,1.120312,1.115888,0.000000,0.000000,0.000000,-0.000626,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 23:44:59.999,1.2311,-0.000081,-1.0,1.230880,-0.000075,-0.000161,45.714286,1.230880,0.000407,1.231695,1.230065,-0.000162,0.000162,-0.000162,0.000406,0.000406
2020-12-30 23:49:59.999,1.2310,-0.000081,-1.0,1.230900,-0.000072,-0.000143,45.714286,1.230900,0.000403,1.231705,1.230095,-0.000081,-0.000162,0.000162,-0.000162,0.000406
2020-12-30 23:54:59.999,1.2309,-0.000081,-1.0,1.230910,-0.000077,-0.000130,47.058824,1.230910,0.000400,1.231710,1.230110,-0.000081,-0.000081,-0.000162,0.000162,-0.000162


In [42]:
cols

['lag1',
 'lag2',
 'lag3',
 'lag4',
 'lag5',
 'SMA',
 'MACD',
 'MACD_Signal',
 'RSI',
 'BB_Upper',
 'BB_Lower']

In [43]:
# Separar los datos en predictores (X) y variable objetivo (y)
X = df[cols]
y = df["direction"]


In [44]:
# Escalar los predictores
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [45]:
# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [46]:
# Entrenar el modelo de regresión logística con pesos balanceados
lm = LogisticRegression(
    multi_class="multinomial", solver="lbfgs", max_iter=10000, class_weight="balanced"
)

In [47]:
# Ajustar el modelo
lm.fit(X_train, y_train)

e:\algo-trading-env\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,10000
,multi_class,'multinomial'


In [48]:
# Predecir en el conjunto de prueba
y_pred = lm.predict(X_test)

In [49]:
# Evaluar el modelo
print("\nReporte de clasificación (Conjunto de prueba):")
print(classification_report(y_test, y_pred, zero_division=0))
print("\nMatriz de confusión (Conjunto de prueba):")
print(confusion_matrix(y_test, y_pred))


Reporte de clasificación (Conjunto de prueba):
              precision    recall  f1-score   support

        -1.0       0.54      0.55      0.54      6723
         0.0       0.52      0.51      0.52      7105
         1.0       0.54      0.54      0.54      6797

    accuracy                           0.53     20625
   macro avg       0.53      0.53      0.53     20625
weighted avg       0.53      0.53      0.53     20625


Matriz de confusión (Conjunto de prueba):
[[3692 1575 1456]
 [1759 3603 1743]
 [1392 1708 3697]]


Claro. Aquí tienes la interpretación de tus resultados:

---

### **Reporte de clasificación**

| Clase | Precision | Recall | F1-score | Support |
|-------|-----------|--------|----------|---------|
| -1.0  |   0.54    |  0.55  |   0.54   |  6723   |
|  0.0  |   0.52    |  0.51  |   0.52   |  7105   |
|  1.0  |   0.54    |  0.54  |   0.54   |  6797   |

- **Precision**: De todas las veces que el modelo predijo una clase, ¿cuántas veces acertó?
- **Recall**: De todas las veces que realmente era esa clase, ¿cuántas veces el modelo la detectó?
- **F1-score**: Media armónica entre precision y recall (balance entre ambos).
- **Support**: Cantidad de muestras reales de cada clase en el conjunto de prueba.

**Conclusión:**  
El modelo tiene un desempeño **similar en todas las clases** (alrededor de 0.53-0.54 en precisión, recall y F1-score).  
La **exactitud global (accuracy)** es 0.53, es decir, el modelo acierta el 53% de las veces, apenas mejor que un clasificador aleatorio para tres clases balanceadas (que sería ~33%).

---

### **Matriz de confusión**



In [ ]:
[[3692 1575 1456]
 [1759 3603 1743]
 [1392 1708 3697]]



- **Filas:** Clase real
- **Columnas:** Clase predicha

Por ejemplo:
- De los **6723** casos reales de clase -1.0:
  - 3692 fueron correctamente clasificados como -1.0
  - 1575 fueron clasificados como 0.0
  - 1456 como 1.0

- De los **7105** casos reales de clase 0.0:
  - 3603 correctamente como 0.0
  - 1759 como -1.0
  - 1743 como 1.0

- De los **6797** casos reales de clase 1.0:
  - 3697 correctamente como 1.0
  - 1392 como -1.0
  - 1708 como 0.0

**Conclusión de la matriz:**  
El modelo acierta más de lo que falla, pero aún hay muchas confusiones entre clases. No hay una clase que esté siendo ignorada completamente, pero tampoco hay una separación clara.

---

### **Resumen general**

- El modelo tiene un desempeño **moderado** (53% de acierto).
- Las tres clases tienen resultados similares, lo que indica que el modelo no está sesgado hacia una sola clase.
- Hay margen de mejora: podrías probar con más predictores, ajustar hiperparámetros, o probar otros modelos.

¿Te gustaría recomendaciones para mejorar el modelo?

In [50]:
pd.Series(y_pred).value_counts()

 1.0    6896
 0.0    6886
-1.0    6843
Name: count, dtype: int64